### Install and Import Necessary Libraries

In [1]:
# Install necessary packages
! pip install -qU langchain-openai langchain openai

In [2]:
import pandas as pd
import getpass
import os

### Set up OpenAI API key

In [3]:
def _get_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_get_env("OPENAI_API_KEY")


#### Function to query the LLM

In [4]:
from openai import OpenAI
import json

client = OpenAI()

def get_answer(question, model):

    prompt = f"""You are an expert in high school mathematics.
    Please solve the following high school math problem step-by-step. Follow these guidelines:
    1. Read the problem carefully and identify the key information.
    2. Write out the given information and what needs to be found.
    3. Choose the appropriate mathematical concept or formula to solve the problem.
    4. Show each step of your calculation clearly, explaining your reasoning.
    5. Double-check your calculations for accuracy.
    6. Provide the final answer clearly.
    7. Do not leave the final in fraction.
    Here is the problem:
    {question}
    Step-by-step solution:"""

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

#### Function to Extract Answer with

In [5]:
import re

def extract_numerical_answer(text):
    # Look for patterns like "Final answer: X" or "The answer is X" at the end of the text
    match = re.search(r'(?:final answer|the answer is)[:\s]*([+-]?\d*\.?\d+)', text, re.IGNORECASE)
    if match:
        return float(match.group(1))
    else:
        # If no clear final answer, look for the last number in the text
        numbers = re.findall(r'[+-]?\d*\.?\d+', text)
        return float(numbers[-1]) if numbers else None

In [6]:
def process_row(row, model):
    problem_id = row['problem_id']
    problem_text = row['problem_text']

    llm_reasoning = get_answer(row, model)  # Changed to pass the entire row

    numerical_answer = extract_numerical_answer(llm_reasoning)

    return {
        'problem_id': problem_id,
        'problem_text': problem_text,
        'llm_reasoning': llm_reasoning,
        'answer': numerical_answer
    }

Using pool executor to make it faster (`max_workers=6`). This controls how many concurrent queries to be sent to OpenAI

In [7]:
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f, model):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el, model)  # Added model parameter
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results


/home/nkama/.pyenv/versions/3.10.6/envs/llmMath/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we create a function for processing each row:

At this point, let's load the train data - so we can use it for evaluation our solution offline (without submitting to Kaggle):

In [8]:
df_train = pd.read_csv('data/train.csv')
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [9]:
rows = df_train.head().to_dict(orient='records')
process_row(rows[0], 'gpt-4o')

{'problem_id': 2374,
 'problem_text': 'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.',
 'llm_reasoning': "To solve the problem \\(\\dfrac{17}{5} \\div \\dfrac{34}{3} + 1.3\\), we need to follow these steps:\n\n### Step 1: Read the Problem Carefully\nThe problem requires us to simplify the given expression \\(\\dfrac{17}{5} \\div \\dfrac{34}{3} + 1.3\\).\n\n### Step 2: Write Out the Given Information\n- Given expression: \\(\\dfrac{17}{5} \\div \\dfrac{34}{3} + 1.3\\)\n\n### Step 3: Choose the Appropriate Mathematical Concept\nWe need to:\n1. Divide the fractions using:\n   \\[\n   \\frac{a}{b} \\div \\frac{c}{d} = \\frac{a}{b} \\times \\frac{d}{c}\n   \\]\n2. Add the result to 1.3.\n\n### Step 4: Show Each Step of Calculation Clearly\n\n#### Step 4.1: Division of Fractions\nFirst, compute the division \\(\\dfrac{17}{5} \\div \\dfrac{34}{3}\\):\n\n\\[\n\\dfrac{17}{5} \\div \\dfrac{34}{3} = \\dfrac{17}{5} \\times \\dfrac{3}{34}\n\\]\n\nNext, multiply the fractio

This is how we combine the two:

In [11]:
results = map_progress(pool, rows, process_row, "gpt-4o-mini")

df_results = pd.DataFrame(results)
df_results

100%|██████████| 5/5 [00:10<00:00,  2.10s/it]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,Let’s follow the guidelines step-by-step to so...,1.6
1,4723,"In a company of 30 people, 25 use the social n...","To solve the problem step-by-step, we'll analy...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,Let's solve the problem step-by-step:\n\n### S...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,Let's solve the problem step-by-step.\n\n### S...,256.0
4,9237,A traveler from Moscow wants to visit four cit...,"To solve the problem, let's follow the guideli...",3.0


In [12]:
from accuracy import score
score(df_train.head(), df_results, 'problem_id')

0.8

Let's evaluate the answer on these 5 examples:

In [13]:
def prepare_prompts_and_get_answers(df, model):
    rows = df.to_dict(orient='records')
    results = map_progress(pool, rows, process_row, model)
    return pd.DataFrame(results)

In [14]:
df_train_results = prepare_prompts_and_get_answers(df_train, "gpt-4o-mini")
df_train_results.head()

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [01:54<00:00,  1.54s/it]

100%|██████████| 100/100 [02:14<00:00,  1.35s/it]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,Let's solve the problem step-by-step.\n\n### S...,1.6
1,4723,"In a company of 30 people, 25 use the social n...","To solve the problem, let’s go through it step...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,Let's solve the problem step-by-step following...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,Let's solve the given problem step-by-step.\n\...,256.0
4,9237,A traveler from Moscow wants to visit four cit...,"To solve this problem, we need to find a combi...",53.0


In [15]:
score(df_train, df_train_results, 'problem_id')

0.94

## Preparing the test submission

In [18]:
df_test = pd.read_csv('data/test.csv')

df_test_results = prepare_prompts_and_get_answers(df_test, "gpt-4o-mini")

submission = df_test_results[['problem_id', 'answer']]
submission.to_csv('starter_notebook_submission.csv', index=False)

100%|██████████| 100/100 [01:54<00:00,  1.15s/it]


 99%|█████████▉| 99/100 [01:52<00:01,  1.10s/it]

In [21]:
df_test_results.head()

,problem_id,problem_text,llm_reasoning,answer
0,11919,Find the value of the expression \(4.8 \cdot 2...,Let's solve the problem step-by-step following...,12.0
1,8513,The plane's navigation system informs the pass...,Let's solve the problem step-by-step:\n\n### S...,315.0
2,7887,The volume of a rectangular parallelepiped is ...,Let's break down the problem step-by-step:\n\n...,4.0
3,5272,Find the root of the equation: $\left(\dfrac{1...,To solve the equation \(\left(\dfrac{1}{8}\rig...,6.0
4,8295,"At the school, there are two-person camping te...","To solve the problem step-by-step, let's follo...",13.0


In [22]:
submission.head()

,problem_id,answer
0,11919,12.0
1,8513,315.0
2,7887,4.0
3,5272,6.0
4,8295,13.0


In [ ]:
# Close the pool after use
#pool.shutdown()